##🎯 Definición de la Problemática
Problema: Identificación de patrones y factores de riesgo de la violencia intrafamiliar en Colombia para optimizar la asignación de recursos y las estrategias de prevención.

Qué: Las entidades gubernamentales y ONGs en Colombia enfrentan el desafío de asignar recursos limitados (líneas de ayuda, refugios, patrullas policiales, programas de apoyo) para combatir la violencia intrafamiliar (VIF) de manera efectiva.

Para Quién: Los principales interesados son las Comisarías de Familia, el ICBF (Instituto Colombiano de Bienestar Familiar), la Policía Nacional, el Ministerio de Salud y las organizaciones de apoyo a víctimas.

Por Qué Requiere Analítica: Un simple conteo nacional de casos no es suficiente. Se requiere analítica de datos para descubrir patrones ocultos y responder preguntas críticas:

¿En qué municipios y departamentos (análisis espacial) se concentran los casos?

¿En qué meses del año o días de la semana (análisis temporal) aumentan los reportes?

¿Cuáles son los perfiles demográficos (género y edad) más vulnerables?

¿Qué tipos de armas o medios son más comunes?

El análisis permite pasar de una estrategia reactiva a una predictiva y preventiva, focalizando las campañas y los recursos en las zonas y poblaciones de mayor riesgo.

##📊 Descripción de Variables Relevantes

| Variable | Descripción | Utilidad para el Caso de Uso |
| :--- | :--- | :--- |
| **`DEPARTAMENTO`** | Nombre del departamento donde ocurrió el hecho. | Identificar los departamentos con mayor incidencia ("puntos calientes"). |
| **`MUNICIPIO`** | Nombre del municipio donde ocurrió el hecho. |  Permite focalizar recursos a nivel local, que es donde se implementan las políticas. |
| **`FECHA HECHO`** | Fecha completa en que ocurrió el incidente. | Crear series de tiempo para ver tendencias anuales, estacionalidad (ej., picos en festividades) o patrones semanales. |
| **`GENERO`** | Género de la víctima. | Identificar el género más afectado. Crucial para diseñar campañas de sensibilización y servicios de apoyo específicos. |
| **`GRUPO ETARIO`** | Rango de edad de la víctima (ej., "ADULTOS", "ADOLESCENTES"). | Identificar las edades más vulnerables y entender si el riesgo cambia con la edad. |
| **`ARMAS MEDIOS`** | Medio o arma utilizada en la agresión (ej., "ARMA BLANCA", "CONTUNDENTES"). |  Entender la gravedad y naturaleza de la violencia. Ayuda a informar a la policía y servicios médicos. |
| **`CANTIDAD`** | Conteo de casos para esa combinación de factores. |  Es la medida principal que se usará para cuantificar la incidencia de la VIF en los diferentes segmentos. |

### Selección del Dataset: Violencia Intrafamiliar en Colombia

**Enlace al Dataset:**
[`https://www.kaggle.com/datasets/oscardavidperilla/domestic-violence-in-colombia`](https://www.kaggle.com/datasets/oscardavidperilla/domestic-violence-in-colombia)

**Explicación de la Selección:**

He seleccionado este dataset porque aborda directamente la problemática definida: la necesidad de entender los patrones de la **Violencia Intrafamiliar (VIF) en Colombia** para una mejor asignación de recursos; este tema me llama la atención por que me gustaría analizar un tema tan delicado en Colombia y como puedo descubrir algunas caracteristicas de violencia en el pais.

La pertinencia del dataset radica en su **alta granularidad**:

1.  **Geográfica:** Contiene datos a nivel de `DEPARTAMENTO` y `MUNICIPIO`, lo cual es esencial para identificar "puntos calientes" y focalizar las intervenciones locales.
2.  **Temporal:** La variable `FECHA HECHO` permite realizar análisis de series de tiempo para detectar estacionalidad (ej., aumento en meses de vacaciones o festividades) y patrones diarios.
3.  **Demográfica:** Las variables `GENERO` y `GRUPO ETARIO` son cruciales para perfilar a las víctimas más vulnerables y diseñar campañas de prevención y apoyo específicas para esos grupos.

Este dataset, proveniente de datos de la Policía Nacional de Colombia, proporciona la materia prima necesaria para construir modelos de Big Data que respondan **dónde**, **cuándo** y a **quién** afecta más la violencia intrafamiliar, permitiendo así una toma de decisiones informada y basada en evidencia.

##Modelo entidad–relación (ERD)

![](ModeloER.png)

##📊 Descripción Diagrama Entidad-Relación (ERD)

### Descripción del modelo

El modelo entidad–relación propuesto se compone de dos tablas principales: **DEPARTAMENTO** e **INCIDENTE**.  
Cada registro de la tabla `INCIDENTE` está vinculado a un único `DEPARTAMENTO` mediante la clave foránea `id_departamento`, estableciendo una relación **uno a muchos**.  

La estructura permite organizar la información geográfica y los datos de cada hecho violento de forma normalizada, facilitando la ejecución de consultas analíticas, como:
- Número de incidentes por departamento o municipio.  
- Análisis de tendencias por fecha, género o grupo etario.  
- Identificación de los medios o armas más utilizados en cada zona.  

Este diseño garantiza integridad referencial y mejora la eficiencia en el manejo del volumen de datos del dataset.


#Cargar el CSV

In [0]:
!pip install kagglehub[pandas-datasets]>=0.3.8

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


Importación de Librerias

In [0]:
import os
import zipfile
import kagglehub
import pandas as pd 

Funciones para Descargar, Extraer y el Leer el Dataset

In [0]:
def download_dataset_zip(url = ""):
        print("Descargando dataset desde Kaggle...")
        dataset_path = kagglehub.dataset_download(url)
        print("Ruta al dataset:", dataset_path)
        return dataset_path
    
def extract_zip_files(dataset_path):
        zip_files = [f for f in os.listdir(dataset_path) if f.endswith('.zip')]
        if zip_files:
            zip_file = os.path.join(dataset_path, zip_files[0])
            extract_dir = os.path.join(dataset_path, "extracted")
            os.makedirs(extract_dir, exist_ok=True)
            print(f"Extrayendo {zip_file} en {extract_dir}...")
            with zipfile.ZipFile(zip_file, "r") as z:
                z.extractall(extract_dir)
            return extract_dir
        else:
            # Si no se encuentra un ZIP, se verifica si existen archivos CSV en la ruta
            csv_files = [f for f in os.listdir(dataset_path) if f.endswith('.csv')]
            if csv_files:
                print("No se encontró archivo ZIP pero se detectaron archivos CSV; se asume que el dataset ya se encuentra extraído.")
                return dataset_path
            else:
                raise FileNotFoundError("No se encontró ningún archivo .zip ni archivos .csv en la ruta del dataset")

def create_csv(csv_dir):
        #os.makedirs('src/static/csv', exist_ok=True)
        csv_files = [f for f in os.listdir(csv_dir) if f.endswith('.csv')]
        if not csv_files:
            raise FileNotFoundError("No se encontraron archivos CSV en el directorio extraído")

        for file in csv_files:
            file_path = os.path.join(csv_dir, file)
            print(f"Leyendo {file_path}...")
            try:
                df = pd.read_csv(file_path, encoding="latin1")
            except Exception as e:
                print(f"Error al leer {file}: {e}")
                continue
            print(f"Creando/actualizando ")
        print("cvs creado correctamente en ")
        return df

Decargar el DataSet

In [0]:
df = pd.DataFrame()
dataset_path = download_dataset_zip("oscardavidperilla/domestic-violence-in-colombia")
csv_dir = extract_zip_files(dataset_path)
df = create_csv(csv_dir)

Descargando dataset desde Kaggle...


100%|██████████| 3.02M/3.02M [00:00<00:00, 37.8MB/s]

Extracting files...
Ruta al dataset: /home/spark-9fb0a7b3-9afd-4156-af54-6d/.cache/kagglehub/datasets/oscardavidperilla/domestic-violence-in-colombia/versions/1
No se encontró archivo ZIP pero se detectaron archivos CSV; se asume que el dataset ya se encuentra extraído.
Leyendo /home/spark-9fb0a7b3-9afd-4156-af54-6d/.cache/kagglehub/datasets/oscardavidperilla/domestic-violence-in-colombia/versions/1/Reporte_Delito_Violencia_Intrafamiliar_Polic_a_Nacional.csv...


Creando/actualizando 
cvs creado correctamente en 


/home/spark-9fb0a7b3-9afd-4156-af54-6d/.ipykernel/4475/command-5993574038886217-2394460318:36: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, encoding="latin1")


Verificación del cargue del DataSet

In [0]:
df.head(4)

,DEPARTAMENTO,MUNICIPIO,CODIGO DANE,ARMAS MEDIOS,FECHA HECHO,GENERO,GRUPO ETARIO,CANTIDAD
0,ATLÃNTICO,BARRANQUILLA (CT),8001000,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,MASCULINO,ADULTOS,1
1,BOYACÃ,DUITAMA,15238000,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,FEMENINO,ADULTOS,1
2,CAQUETÃ,PUERTO RICO,18592000,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,MASCULINO,ADULTOS,1
3,CASANARE,MANÃ,85139000,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,FEMENINO,ADULTOS,1


Conversión del DataFrame de Pands a Spark

In [0]:
import numpy as np

df['CODIGO DANE'] = df['CODIGO DANE'].replace('NO REPORTA', np.nan)
df['CODIGO DANE'] = df['CODIGO DANE'].astype('float64')


spark_df = spark.createDataFrame(df)

display(spark_df.head(10))

DEPARTAMENTO,MUNICIPIO,CODIGO DANE,ARMAS MEDIOS,FECHA HECHO,GENERO,GRUPO ETARIO,CANTIDAD
ATLÃNTICO,BARRANQUILLA (CT),8001000.0,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,MASCULINO,ADULTOS,1
BOYACÃ,DUITAMA,1.5238E7,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,FEMENINO,ADULTOS,1
CAQUETÃ,PUERTO RICO,1.8592E7,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,MASCULINO,ADULTOS,1
CASANARE,MANÃ,8.5139E7,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,FEMENINO,ADULTOS,1
CUNDINAMARCA,BOGOTÃ D.C. (CT),1.1001E7,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,FEMENINO,ADULTOS,1
SUCRE,SINCELEJO (CT),7.0001E7,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,FEMENINO,ADULTOS,1
VALLE,CALI (CT),7.6001E7,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,FEMENINO,ADULTOS,2
VALLE,CALI (CT),7.6001E7,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,MASCULINO,ADULTOS,2
VALLE,TULUÃ,7.6834E7,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,FEMENINO,ADULTOS,1
HUILA,PITALITO,4.1551E7,ARMA DE FUEGO,1/01/2010,FEMENINO,ADULTOS,1


Crear vista temporal

In [0]:
spark_df.createTempView("tbl_violencia_colombia")
display(spark_df.head(10))

DEPARTAMENTO,MUNICIPIO,CODIGO DANE,ARMAS MEDIOS,FECHA HECHO,GENERO,GRUPO ETARIO,CANTIDAD
ATLÃNTICO,BARRANQUILLA (CT),8001000.0,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,MASCULINO,ADULTOS,1
BOYACÃ,DUITAMA,1.5238E7,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,FEMENINO,ADULTOS,1
CAQUETÃ,PUERTO RICO,1.8592E7,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,MASCULINO,ADULTOS,1
CASANARE,MANÃ,8.5139E7,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,FEMENINO,ADULTOS,1
CUNDINAMARCA,BOGOTÃ D.C. (CT),1.1001E7,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,FEMENINO,ADULTOS,1
SUCRE,SINCELEJO (CT),7.0001E7,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,FEMENINO,ADULTOS,1
VALLE,CALI (CT),7.6001E7,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,FEMENINO,ADULTOS,2
VALLE,CALI (CT),7.6001E7,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,MASCULINO,ADULTOS,2
VALLE,TULUÃ,7.6834E7,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,FEMENINO,ADULTOS,1
HUILA,PITALITO,4.1551E7,ARMA DE FUEGO,1/01/2010,FEMENINO,ADULTOS,1


Verificación de la creación de la tabla y conteo de registros

In [0]:
%sql
select count(*) from tbl_violencia_colombia

count(*)
476970


# Creación de la tabla e inserción de datos

Creación de las tablas en la BD para DEPARTAMENTO e INCIDENTE

In [0]:
%sql
-- Crear tabla DEPARTAMENTO
CREATE OR REPLACE TABLE departamento (
  id_departamento INT,
  nombre STRING,
  codigo_dane STRING,
  PRIMARY KEY (id_departamento)
);

-- Crear tabla INCIDENTE
CREATE OR REPLACE TABLE incidente (
  id_incidente INT,
  id_departamento INT,
  municipio STRING,
  fecha_hecho DATE,
  genero STRING,
  grupo_etario STRING,
  armas_medios STRING,
  cantidad BIGINT,
  FOREIGN KEY (id_departamento) REFERENCES departamento(id_departamento)
);


In [0]:
spark_df = spark.createDataFrame(df)

In [0]:
# Renombrar las columnas para que no contengan espacios ni caracteres especiales

col_count = {}
new_columns = []
for col in spark_df.columns:
    new_col = col.replace(" ", "_").replace(",", "_").replace(";", "_") \
        .replace("{", "_").replace("}", "_").replace("(", "_") \
        .replace(")", "_").replace("\n", "_").replace("\t", "_") \
        .replace("=", "_")
    if new_col in col_count:
        col_count[new_col] += 1
        new_col = f"{new_col}_{col_count[new_col]}"
    else:
        col_count[new_col] = 0
    new_columns.append(new_col)

spark_df = spark_df.toDF(*new_columns)

display(spark_df.head(10))

DEPARTAMENTO,MUNICIPIO,CODIGO_DANE,ARMAS_MEDIOS,FECHA_HECHO,GENERO,GRUPO_ETARIO,CANTIDAD
ATLÃNTICO,BARRANQUILLA (CT),8001000.0,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,MASCULINO,ADULTOS,1
BOYACÃ,DUITAMA,1.5238E7,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,FEMENINO,ADULTOS,1
CAQUETÃ,PUERTO RICO,1.8592E7,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,MASCULINO,ADULTOS,1
CASANARE,MANÃ,8.5139E7,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,FEMENINO,ADULTOS,1
CUNDINAMARCA,BOGOTÃ D.C. (CT),1.1001E7,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,FEMENINO,ADULTOS,1
SUCRE,SINCELEJO (CT),7.0001E7,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,FEMENINO,ADULTOS,1
VALLE,CALI (CT),7.6001E7,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,FEMENINO,ADULTOS,2
VALLE,CALI (CT),7.6001E7,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,MASCULINO,ADULTOS,2
VALLE,TULUÃ,7.6834E7,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,FEMENINO,ADULTOS,1
HUILA,PITALITO,4.1551E7,ARMA DE FUEGO,1/01/2010,FEMENINO,ADULTOS,1


In [0]:
spark.sql("DROP TABLE IF EXISTS tbl_violencia_colombia")

spark_df.write.mode("overwrite").saveAsTable("tbl_violencia_colombia")

Verificar la tabla creada

In [0]:
%sql
SELECT * 
FROM tbl_violencia_colombia
LIMIT 5;

DEPARTAMENTO,MUNICIPIO,CODIGO_DANE,ARMAS_MEDIOS,FECHA_HECHO,GENERO,GRUPO_ETARIO,CANTIDAD
ATLÃNTICO,BARRANQUILLA (CT),8001000.0,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,MASCULINO,ADULTOS,1
BOYACÃ,DUITAMA,1.5238E7,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,FEMENINO,ADULTOS,1
CAQUETÃ,PUERTO RICO,1.8592E7,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,MASCULINO,ADULTOS,1
CASANARE,MANÃ,8.5139E7,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,FEMENINO,ADULTOS,1
CUNDINAMARCA,BOGOTÃ D.C. (CT),1.1001E7,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,FEMENINO,ADULTOS,1


4️⃣ Evidencia con consultas SQL
1. Conteo de registros

In [0]:
%sql
SELECT COUNT(*) FROM tbl_violencia_colombia;


COUNT(*)
476970


### Interpretación de resultados

La tabla **tbl_violencia_colombia** contiene un total de **476,970 registros**.  
Esto indica que el dataset cuenta con una gran cantidad de casos reportados de violencia doméstica en Colombia, lo que permitirá realizar análisis detallados por variables como tipo de violencia, año, género, y ubicación geográfica.  
El tiempo de ejecución (~27 segundos) sugiere que se está trabajando con un volumen de datos considerable, ideal para aplicar consultas analíticas más avanzadas.


2. Nombres y tipos de columnas:


In [0]:
%sql
DESCRIBE TABLE tbl_violencia_colombia;

col_name,data_type,comment
DEPARTAMENTO,string,null
MUNICIPIO,string,null
CODIGO_DANE,string,null
ARMAS_MEDIOS,string,null
FECHA_HECHO,string,null
GENERO,string,null
GRUPO_ETARIO,string,null
CANTIDAD,bigint,null


### Interpretación de resultados

La tabla **tbl_violencia_colombia** cuenta con **8 columnas**, cuyos tipos de datos se distribuyen principalmente en formato `string`, salvo la columna **CANTIDAD**, que está definida como `BIGINT`.  
Esto indica que la información almacenada corresponde mayormente a variables categóricas o de texto (como ubicación, género o tipo de arma), mientras que **CANTIDAD** representa un valor numérico utilizado para cuantificar los casos reportados.  
El conocimiento de los tipos de datos es fundamental para definir correctamente las operaciones de agregación, filtrado y análisis posteriores.


3. Consulta con filtro

In [0]:
%sql
SELECT * FROM tbl_violencia_colombia WHERE DEPARTAMENTO = 'ANTIOQUIA' LIMIT 20;

DEPARTAMENTO,MUNICIPIO,CODIGO_DANE,ARMAS_MEDIOS,FECHA_HECHO,GENERO,GRUPO_ETARIO,CANTIDAD
ANTIOQUIA,ANORÃ,5040000.0,CONTUNDENTES,27/09/2017,MASCULINO,ADULTOS,1
ANTIOQUIA,APARTADÃ,5045000.0,CONTUNDENTES,27/09/2017,FEMENINO,ADULTOS,2
ANTIOQUIA,CAUCASIA,5154000.0,CONTUNDENTES,27/09/2017,FEMENINO,ADULTOS,1
ANTIOQUIA,EL CARMEN DE VIBORAL,5148000.0,CONTUNDENTES,27/09/2017,FEMENINO,ADULTOS,1
ANTIOQUIA,ENVIGADO,5266000.0,CONTUNDENTES,27/09/2017,FEMENINO,ADULTOS,1
ANTIOQUIA,ENVIGADO,5266000.0,CONTUNDENTES,27/09/2017,MASCULINO,ADULTOS,1
ANTIOQUIA,MEDELLÃN (CT),5001000.0,CONTUNDENTES,27/09/2017,FEMENINO,ADULTOS,2
ANTIOQUIA,MEDELLÃN (CT),5001000.0,NO REPORTADO,27/09/2017,FEMENINO,ADULTOS,1
ANTIOQUIA,ANDES,5034000.0,SIN EMPLEO DE ARMAS,27/09/2017,FEMENINO,ADULTOS,1
ANTIOQUIA,ARGELIA,5055000.0,SIN EMPLEO DE ARMAS,27/09/2017,FEMENINO,MENORES,1


### Interpretación de resultados

La consulta filtra los registros de la tabla **tbl_violencia_colombia** para mostrar únicamente los casos ocurridos en el departamento de **Antioquia**.  
El resultado arroja más de **10,000 registros** en este caso solo usamos la muestra limite de 20 registros para no saturar el notebook, lo cual evidencia que este departamento presenta un volumen considerable de reportes de violencia doméstica dentro del dataset.  
Entre los campos visibles se observan variables como el municipio, tipo de arma o medio utilizado, fecha del hecho, género y grupo etario de las víctimas, así como la cantidad de casos.  
Esta información permitirá posteriormente realizar análisis comparativos entre municipios, identificar tendencias temporales y examinar la distribución de víctimas por edad y género en la región.


### Conclusión de la sección

Las consultas ejecutadas confirman que la base de datos fue creada correctamente, las tablas tienen la estructura esperada y la información se encuentra disponible para realizar análisis exploratorios y consultas agregadas.  
Estas evidencias son fundamentales para validar la calidad del modelo y garantizar la confiabilidad de los resultados obtenidos en etapas posteriores.
